In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,0,40.0,0.0,1400.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,63.0
1,1,48.0,4.0,2850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,86.0
2,2,39.0,0.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
3,3,46.0,5.0,1400.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0
4,4,43.0,1.0,1500.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0


In [4]:
test.index = test.Id

In [5]:
test.drop(columns = 'Id', inplace=True)

In [6]:
train.drop(columns = 'Id', inplace=True)

In [7]:
y = train['mean_exam_points']
#y_log = np.log1p(y)
X = train.drop('mean_exam_points', axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    train_size=0.8, test_size=0.2, random_state=48)

In [8]:
from sklearn.tree import DecisionTreeRegressor

In [9]:
# Разбиение датасета в узле
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [10]:
# Расчёт дисперсии значений (вместо gini)
def dispersion(labels):
    return np.std(labels)


# Расчет качества для задачи регрессии
def quality_regression(left_labels, right_labels, current_dispersion):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_dispersion - p * dispersion(left_labels) - (1 - p) * dispersion(right_labels)


# аналог classify_object для регрессии
def predict_object(obj, node):
    
    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction_regression
        return answer

    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)

    
# аналог predict_class для регрессии
def predict_value(data, tree):
    
    val = []
    for obj in data:
        prediction = predict_object(obj, tree)
        val.append(prediction)
    return np.array(val)
    

# Нахождение наилучшего разбиения для задачи регрессии
def find_best_split_regression(data, labels):
    
       #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_dispersion = dispersion(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality_regression(true_labels, false_labels, current_dispersion)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index


# Построение дерева регрессии с помощью рекурсивной функции
def build_tree_regression(data, labels, tree_depth=1, max_depth=50, min_samples_split = 2):
    
    quality, t, index = find_best_split_regression(data, labels)

    #  Базовый случай 1 - прекращаем рекурсию, когда нет прироста в качества
    #if quality == 0:
     #   return Leaf(data, labels)
    
    # Базовый случай 2 - прекращаем рекурсию, когда достигнута максимальная глубина дерева
    if tree_depth >= max_depth:
        return Leaf(data, labels)
    
    # Базовый случай 3 - прекращаем рекурсию, когда в узле осталось количество сэмплов меньше, чем необходимое
    if len(labels) <= min_samples_split:
        return Leaf(data, labels)

    # Увеличиваем глубину дерева на 1
    tree_depth += 1

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree_regression(true_data, true_labels, tree_depth, max_depth, min_samples_split)
    false_branch = build_tree_regression(false_data, false_labels, tree_depth, max_depth, min_samples_split)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [11]:
# Реализуем класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [12]:
# И класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction_classification = self.predict()
        self.prediction_regression = self.predict_reg()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction  

    def predict_reg(self):
        #  найдем значение как среднее по выборке   
        prediction = np.mean(self.labels)
        return prediction  

In [13]:
def deriv(y, z):
    return 2*(y - z)

In [14]:
def mean_squared_error_1(y_real, prediction):
    return np.mean((y_real - prediction)**2)

In [15]:
# def gb_predict_1(X, trees_list, coef_list, eta):
#     result = np.zeros(X.shape[0])
#     cnt = 0
#     #print(cnt)
#     for tree in trees_list:
#         #result += eta*coef_list[cnt]*tree.predict(X)
#         result += eta*coef_list[cnt]*predict_value(X.to_numpy(), tree)
#         cnt += 1
#     return result

In [16]:
# def gb_fit_1(n_trees, max_depth, X_train, y_train, coefs, eta):
    
#     # Деревья будем записывать в список
#     trees = []
    
#     for i in range(n_trees):
#         #tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
        
        

#         # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
#         # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
#         if i == 0:
#             # обучаем первое дерево на обучающей выборке
#             tree = build_tree_regression(X_train.to_numpy(), y_train.to_numpy(), 
#                                          max_depth = max_depth, min_samples_split = 12)
#             #tree.fit(X_train, y_train)
#             #trees.append(tree)
            
#         else:
#             # Получим ответы на текущей композиции
#             pred = gb_predict_1(X_train, trees, coefs, eta)
            
#             # алгоритмы начиная со второго обучаем на сдвиг
#             tree = build_tree_regression(X_train.to_numpy(), deriv(y_train.to_numpy(), pred), 
#                                          max_depth = max_depth, min_samples_split = 12)
#             #tree.fit(X_train, deriv(y_train, pred))


#         trees.append(tree)
        
#     return trees

In [17]:
def get_r2(x, y):
    return 1 - ((np.sum((x - y)**2))/(np.sum((y - np.mean(y))**2)))

In [18]:
# eta = 0.015
# n_trees = 250
# coefs = np.ones(n_trees + 1)
# max_depth = 4
# trees = gb_fit_1(n_trees, max_depth, X_train, y_train, coefs, eta)

In [19]:
# Построим дерево по обучающей выборке
my_tree = build_tree_regression(X_train.to_numpy(), y_train.to_numpy(), max_depth = 9, min_samples_split = 10)

In [20]:
res_train = predict_value(X_train.to_numpy(), my_tree)

In [21]:
r2 = get_r2(res_train, y_train)

In [22]:
r2

0.7972923323186527

In [23]:
res_valid = predict_value(X_valid.to_numpy(), my_tree)

In [24]:
r2 = get_r2(res_valid, y_valid)

In [25]:
r2

0.780950871238488

In [26]:
my_tree = build_tree_regression(X.to_numpy(), y.to_numpy(), max_depth = 8, min_samples_split = 12)

In [27]:
res_test = predict_value(test.to_numpy(), my_tree)

In [28]:
output = pd.DataFrame({'Id': test.index,
                       'mean_exam_points': res_test})
output.to_csv('submission.csv', index=False)